# 文字分析

自然語言處理 (NLP) 是處理書面和口語語言之人工智慧 (AI) 的分支。您可以使用 NLP 來組建從文字或語音中擷取語意的解決方案，或在自然語言中制訂有意義之回應的解決方案。

Microsoft Azure *認知服務*包含*文字分析*服務，該服務提供立即可用的 NLP 功能，包括識別文字中的關鍵片語以及根據情緒分類文字。

![一個傀儡程式正在讀取 Notebook](./images/NLP.jpg)

例如，假設虛構組織 *Margie's Travel*鼓勵客戶提交關於旅館住宿的評論。您可以使用文字分析服務透過擷取關鍵片語的方式來摘要評論，判斷哪些評論是好評，哪些是差評，或分析提及已知實體 (例如位置或人) 的評論文字。

## 檢視評論文件

我們首先來看一看客戶給旅館留下的一些評論。

評論在文字檔中。若要查看評論，只需透過按一下儲存格左側的 **[執行儲存格]** (&#9655;) 按鈕來執行下面的程式碼。

In [ ]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## 建立認知服務資源

若要分析這些評論中的文字，您可以使用**文字分析**認知服務。若要使用這些評論，您需要在您的 Azure 訂用帳戶中建置一個**文字分析**或**認知服務**資源 (如果這是您計劃使用或您想要另行追蹤其使用方式的唯一服務，請使用文字分析資源；否則，您可以使用認知服務資源將文字分析服務和其它認知服務組合，確保開發人員使用單一端點和金鑰來存取它們。)

如果您還沒有該資源，請使用以下步驟在您的 Azure 訂用帳戶中建立一個**認知服務**資源：

> **備註**：若您已經有認知服務資源，只需在 Azure 入口網站中開啟其 **[快速入門]** 頁面並將其金鑰和端點複製到下面的儲存格。否則，可追隨下面的步驟來建立一個認知服務資源。

1. 在其它瀏覽器索引標籤中，透過 https://portal.azure.com 開啟 Azure 入口網站，並用您的 Microsoft 帳戶登入。
2. 按一下 **[&#65291; 建立資源]** 按鈕，*搜尋認知服務*，並建立包含以下設定的**認知服務**資源：
    - **訂用帳戶**： *您的 Azure 訂用帳戶*。
    - **資源群組**： *選取或建立具有唯一名稱的資源群組*。
    - **區域**： *選擇任一可用區域*：
    - **名稱**： *輸入唯一名稱*。
    - **定價層**：S0
    - **我確認已閱讀通知並理解通知內容**：已選取。
3. 等待部署完成。然後前往您的認知服務資源，在 **[概觀]** 頁面上，按一下連結以管理服務金鑰。您將需要端點和金鑰，以便從用戶端應用程式連線到您的認知服務資源。

### 獲取您的認知服務資源之金鑰和端點

若要使用您的認知服務資源，用戶端應用程式需要其端點和驗證金鑰：

1. 在 Azure 入口網站中，您的認知服務資源之 **[金鑰和端點]** 頁面上，複製您的資源之**金鑰 1** 並將其貼上到下面的程式碼，取代 **YOUR_COG_KEY**。
2. 複製您的資源之**端點**並將其貼上到下面的程式碼，代替 **YOUR_COG_ENDPOINT**。
3. 透過按一下其綠色 <span style="color:green">&#9655;</span> 按鈕執行下面的儲存格中的程式碼。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## 偵測語言
我們首先識別書寫評論所使用的語言。

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## 擷取關鍵片語

現在您可以分析客戶評論中的文字以識別可以指示主要論題的關鍵片語。

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

關鍵片語可以幫助您理解每條評論中最重要的論題。例如，包含片語「員工樂於助人」或「服務差」的評論可以給您指示評論者的一些主要關注點。

## 判斷情緒

根據情緒*分數*將評論分類為*好評*或*差評*可能會有用。此外，您可以使用文字分析服務來實現這一目標。

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## 擷取已知實體

*實體*是可能會在文字中提及的事物，涉及一些常見的項目類型。例如，位置、人或日期。我們假設您對評論中提及的日期和地點感興趣，您可以使用以下程式碼找到它們。

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

請注意，某些較為知名的實體擁有相關聯的維基百科頁面，在這種情況下，文字分析服務會退回該頁面的 URL。

## 瞭解更多資訊

有關文字分析服務的更多資訊，請查看[文字分析服務文件](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)